# Libraries and Packages

In [ ]:
import lasio
from os import path
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib as mpl
from matplotlib import pyplot as plt

import sklearn
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from scipy.stats import probplot, pearsonr
from warnings import filterwarnings
from colorama import Fore

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
filterwarnings('ignore')

#setting defaults
pd.options.display.max_columns = 15
pd.options.display.max_rows = 40
# plt.style.use('ggplot')
sns.set_style('darkgrid')

In [ ]:
print(f'pandas version : {pd.__version__}')
print(f'numpy version : {np.__version__}')
print(f'scikit learn version : {sklearn.__version__}')
print(f'seaborn version : {sns.__version__}')
print(f'matplotlib version : {mpl.__version__}')
print(f'lasio version : {lasio.version()}')

# Functions

In [ ]:
#select columns to use in the dataframe
col = ['DEPT:1','GR', 'RT', 'RHOB', 'SP', 'CALI', 'PHI']

def fillArbitrary(dfs:pd.DataFrame) -> 'pd.Series|pd.DataFrame':
    """
   fill in missing values in the dataframes with mean value
    """
    df = dfs.copy()
    df.dropna(axis='index', subset=['PHI'], inplace=True)
    for c in df.columns:
        df[c].fillna(df[c].mean(), inplace=True)
        
    depth = df['DEPTH'] #new depth range after dropping rows where nphi and sp are null
    
    df.drop(['DEPTH'], axis='columns', inplace=True) #dropping poorly correlating features
    
    return df, depth

def processData(well:pd.DataFrame, cols:list, name:str) -> 'tuple|pd.DataFrame':
    
    '''
    cleans and process the train dataframe for null values
    
    '''
#     column = cols[:-2]
    df = well.filter(cols, axis='columns')
    
    #filtering odd values in features 
    df['SP'] = np.where(df['SP'] < 0, np.nan, df['SP'])
    df['PHI'] = np.where(df['PHI'] < 0, np.nan, df['PHI'])
    #rename column
    df = df.rename({'DEPT:1':'DEPTH'}, axis='columns')
    df['RT'] = np.log10(df['RT']) #transforms the RT log to logarithmic scale
    
    #filter data to region where PHI readings are available
    if name == 'WellB':
        df = df[(df['DEPTH'] >= 2500) & (df['DEPTH'] <= 4200)]
    else:
        df = df[(df['DEPTH'] >= 3000) & (df['DEPTH'] <= 3600)]
    
    #calling the arbitrtary fill function
    newdf, depth = fillArbitrary(df)
    newdf['DEPTH'] = depth #adding new depth sequence to df
    newdf = newdf.reset_index(drop=True).sort_values('DEPTH')

    return newdf, df.shape

def qq_plot_historesids(y_true, y_pred, name):
    '''
    quantile-quantile and histogram residual plots
    '''
    sns.set_style('darkgrid')
    plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    plt.suptitle(f'Metrics Plot for {name}', fontsize=20)
    
    plt.subplot(1, 2, 1)
    resids = np.subtract(y_true.reshape(-1,1), y_pred.reshape(-1,1))
    sns.distplot(resids)
    plt.title(f'Histogram of residuals for {name}')
    plt.xlabel('Residual value')
    plt.ylabel('count')

    plt.subplot(1, 2, 2)
    resids = np.subtract(y_true, y_pred)
    r2 = round(r2_score(y_true, y_pred), 2)
    r2_adj = round(r2 - (6 - 1)/(y_true.shape[0] - 6) * (1 - r2), 2)
    probplot(resids.flatten(), plot = plt)
    plt.title(f'Residuals vs. Prediction for {name}')
    plt.text(-2, 5, 'Adjusted R2 = ' + r2_adj.astype(str), fontsize=10, c='red')
    plt.ylabel('Residuals')
    plt.xlabel('Predicted Values')

    
def chatterjeeCorr(df:pd.DataFrame, x:str, y:str) -> float:
    '''
    implementing chatterjee method of calculating non-linear relationship between 
    independent and dependent variables
    '''
    dfs = df.copy()
    dfs['x_rk'] = dfs[x].rank()
    dfs['y_rk'] = dfs[y].rank()
    dfs = dfs.sort_values('x_rk')
    sum_term = dfs['y_rk'].diff().abs().sum()
    chatt_corr = (1 - 3 * sum_term / (pow(dfs.shape[0], 2) - 1))
    return chatt_corr


def plotScatter(df:pd.DataFrame, *cols:tuple, y='PHI', rel='linear'):
    
    sns.set_style('whitegrid')
    plt.subplots(nrows=2, ncols=3, figsize=(22, 11))
    plt.suptitle(f'Scatter Plot Relationship', fontsize=20)
    for i, x in enumerate(cols):
        plt.subplot(2, 3, i+1)
        if x == 'RT':
            plt.title(y + ' vs. log' + x)
        else:
            plt.title(y + ' vs. ' + x)
        sns.regplot(x=x, y=y, data=df, color='blue',
                   fit_reg=True, scatter_kws={'s':10, 'alpha':0.5})
        
        if rel == 'linear':
            r, p_val = pearsonr(df[x], df[y])
        elif rel == 'non-linear':
            r = chatterjeeCorr(df, x, y)
        if x == 'DEPTH':
            plt.text(2600, 60, f'R = {r:.2f}', bbox=dict(facecolor='red', alpha=0.5))
        elif x=='CALI':
            plt.text(10, 60, f'R = {r:.2f}', bbox=dict(facecolor='red', alpha=0.5))
        else:
            plt.text(2, 60, f'R = {r:.2f}', bbox=dict(facecolor='red', alpha=0.5))
            

def log_plot(logs, x1, x2, x3, x4, x5, x6, x7, well_name, top, bot):
    
    '''
    plot well logs along with the predicted and ground truth of porosity
    '''
    sns.set_style('white')
    # ztop = logs.DEPTH.min(); zbot=logs.DEPTH.max()
    
#     logs['litho number'] = [1 if i >= (logs['GR'].max() - logs['GR'].min())/2 else 0 for i in logs['GR'] ]
  
    # color = ['black', 'red', 'blue', 'green', 'cyan', 'purple', 'purple']
    title = ['GR(gAPI)', 'log(RT)(ohm-m)', 'RHOB(g/cm3)', 'SP(mV)', 'CALI(in)', 'PHI(m3/m3)', 'Pred PHI(m3/m3)']
    
    f, ax = plt.subplots(nrows=1, ncols=len(title), figsize=(14, 7))
    f.suptitle(f'Curve Plotting for {well_name}', fontsize=20, y=1.05)
    
    ax[0].plot(logs[x1], logs.DEPTH, 'black'); ax[1].semilogx(logs[x2], logs.DEPTH, 'black')
    ax[2].plot(logs[x3], logs.DEPTH, 'black'); ax[3].plot(logs[x4], logs.DEPTH,'black')
    ax[4].plot(logs[x5], logs.DEPTH, 'black'); ax[5].plot(logs[x6], logs.DEPTH, 'green')
    ax[6].plot(logs[x7], logs.DEPTH,'blue')

   
    for i in range(len(ax)):
        ax[i].set_ylim(top,bot); ax[i].invert_yaxis()
        if i != 1:
            ax[i].locator_params(axis='x', nbins=3)
        ax[i].xaxis.label.set_color('black')
        ax[i].grid(True); ax[i].tick_params(axis='x', colors='black')
        ax[i].spines['top'].set_edgecolor('black'); ax[i].set_title(title[i], pad=15)
        ax[i].spines["top"].set_position(("axes", 1.02)); ax[i].xaxis.set_ticks_position("top")
        ax[i].xaxis.set_label_position("top")
    

    ax[0].set_xlim(logs[x1].min(), logs[x1].max()); ax[1].set_xlim(logs[x2].min(), logs[x2].max())
    ax[2].set_xlim(logs[x3].min(), logs[x3].max()); ax[3].set_xlim(logs[x4].min(), logs[x4].max())
    ax[4].set_xlim(logs[x5].min(), logs[x5].max()); ax[5].set_xlim(logs[x6].min(), logs[x6].max())
    ax[6].set_xlim(logs[x6].min(), logs[x6].max());
    
def plot_scatter(df, well_name):    
    '''
    plots the predicted and true poro reading on scatter plot
    '''
    sns.set_style('darkgrid')
    plt.figure(figsize=(5,5.5))
    sns.scatterplot('PHI', 'Predicted PHI', data=df)
    # if well_name
    plt.plot([10, 60], [10, 60], '--', c='black')
    plt.xlabel('True PHI')
    plt.ylabel('Pred PHI')
    plt.title(f'{well_name}')

# Model Architecture

In [ ]:
class BaseModel(object):
    '''
    class to porosity prediction
    '''
    def __init__(self, train, test):
        '''
        takes in the train and test dataframe
        '''
        self.train = train
        self.test = test
        
    def __call__(self, plot=True):
        
        return self._fit_predict(plot=plot)
    
    def describe(self, train):
        '''
        returns the new summary statistics of the dataframe
        '''
        if train:
            return self.train.describe()
        else: 
            return self.test.describe()
    
    def _preprocess(self, train, test):
        
        '''
        desc: splits the train dataframe into 80% train and 20% test and scales the data
        returns: robust-scaled numpy data
        '''

        #getting feature and target
        train_feature = (self.train.drop('PHI', axis='columns')).values;
        test_feature = (self.test.drop('PHI', axis='columns')).values
        trainlabel = np.array((self.train['PHI'])/100);
        testlabel = np.array((self.test['PHI'])/100)
        
        ## Randomly sample cases to create independent training and validation data
        np.random.seed(9988)
        indx = range(train_feature.shape[0])
        indx = train_test_split(indx, test_size = 0.15)
        x_train = train_feature[indx[0],:]; y_train = np.ravel(trainlabel[indx[0]])
        x_test = train_feature[indx[1],:]; y_test = np.ravel(trainlabel[indx[1]])

        #standardization for train and test
        scaler = RobustScaler(quantile_range=(25.0, 75.0)).fit(x_train)
        x_train = scaler.transform(x_train); x_test = scaler.transform(x_test)
        trainfeature = scaler.transform(train_feature); testfeature = scaler.transform(test_feature)
        
        return x_train, x_test, y_train, y_test, trainfeature, trainlabel, testfeature, testlabel
    
    def _fit_predict(self, plot=True):
        
        '''
        fits models on the data and returns the stacked PHI results
        '''
        x_train, x_test, y_train, y_test, trainfeature, trainlabel, testfeature, testlabel = self._preprocess(self.train, self.test)
        
        model1 = SVR(C=10, epsilon=0.1, kernel='rbf', gamma='auto', tol=1e-10)
        model2 = MLPRegressor(hidden_layer_sizes=(5,), activation='relu', solver='adam',
                                   random_state=300, alpha=1, validation_fraction=0.3)
        #fitting model and prediction 
        model1.fit(x_train, y_train); model2.fit(x_train, y_train)
        
        #validation on well one
        print(Fore.BLUE + 'Validation Results on Well 1')
        print('-'*50)
        
        y_pred1_1 = (model1.predict(trainfeature))*100 #svr prediction
        y_pred1_2 = (model2.predict(trainfeature))*100 #mlp prediction
        trainlabel = trainlabel*100 #denormalise the train label
        final1 = (y_pred1_1 + y_pred1_2)/2 # avg. prediction        
        print('The R2-score of SVR for Well 1 : %.2f' %r2_score(trainlabel, y_pred1_1))
        print('The R2-score of MLP for Well 1 : %.2f' %r2_score(trainlabel, y_pred1_2))
        print('The Abso. Error of SVR for Well 1 : %.2f' %mean_absolute_error(trainlabel, y_pred1_1))
        print('The Abso. Error of MLP for Well 1 : %.2f' %mean_absolute_error(trainlabel, y_pred1_2))
        print('The Average R2 Score for Well 1 : %.2f' %r2_score(trainlabel, final1))
        print('The Average Abso. Error for Well 1 : %.2f' %mean_absolute_error(trainlabel, final1))
        print('The Average R for Well 1 : %.2f' %np.sqrt(r2_score(trainlabel, final1)))

        
        print()
        #validation on well two
        print(Fore.YELLOW + 'Validation Results on Well 2')
        print('-'*50)
        y_pred2_1 = (model1.predict(testfeature))*100 #svr prediction
        y_pred2_2 = (model2.predict(testfeature))*100 #mlp prediction
        testlabel = testlabel*100 #denormalise the validation label
        final2 = (y_pred2_1 + y_pred2_2)/2 # avg. prediction
        print('The R2-score of SVR for Well 2 : %.2f' %r2_score(testlabel, y_pred2_1))
        print('The R2-score of MLP for Well 2 : %.2f' %r2_score(testlabel, y_pred2_2))
        print('The Abso. Error of SVR for Well 2 : %.2f' %mean_absolute_error(testlabel, y_pred2_1))
        print('The Abso. Error of MLP for Well 2 : %.2f' %mean_absolute_error(testlabel, y_pred2_2))
        print('The Average R2 Score for Well 2 : %.2f' %r2_score(testlabel, final2))
        print('The Average Abso. Error for Well 2 : %.2f' %mean_absolute_error(testlabel, final2))
        print('The Average R for Well 2 : %.2f' %np.sqrt(r2_score(testlabel, final2)))
        
        if plot:
            #plot qq-plot and histogram of residuals
            qq_plot_historesids(trainlabel, final1, 'WELLA#')
            qq_plot_historesids(testlabel, final2, 'WELLB#')


        return final1, final2

# Data Loading

In [ ]:
# get all paths and alphabetically ordered
# paths = sorted(glob.glob(os.path.join("./", "*.LAS")))
paths = ['WELLB#.las', 'WELLA#.las']
well_df = [0] * 2

for i in range(len(paths)):
    # read with lasio and convert to dataframe
    df = (lasio.read(path.join('./', paths[i]))).df()

    well_df[i] = df.reset_index()

well1, well2 = well_df

well1, shape1=processData(well1, col, name='WellA') #validation
well2, shape2=processData(well2, col, name='WellB') #training

train = well2
test = well1

# Correlation

In [ ]:
plt.figure(figsize=((13, 7)))
corr = train.select_dtypes('float64').corr()
sns.heatmap(corr, annot=True, vmin=0, vmax=1, linewidths=0.01)

In [ ]:
plotScatter(train, 'DEPTH', 'GR', 'RHOB', 'CALI', 'SP', 'RT', rel='non-linear') #chatterjee
plotScatter(train, 'DEPTH', 'GR', 'RHOB', 'CALI', 'SP', 'RT', rel='linear') #pearson's

# Prediction

In [ ]:
basemodel = BaseModel(train, test)
well1_pred, well2_pred = basemodel(plot=True)

#add predicted results to the original dataframes
train['Predicted PHI'] = well1_pred
test['Predicted PHI'] = well2_pred

# Plots

In [ ]:
#validation scatter plots
plot_scatter(train, 'WellA#')
plot_scatter(test, 'WellB#')

In [ ]:
#log plots
log_plot(train, 'GR', 'RT', 'RHOB', 'SP', 'CALI', 'PHI', 'Predicted PHI', well_name='WellA#',top=2500, bot=2700)
log_plot(test, 'GR', 'RT', 'RHOB', 'SP', 'CALI', 'PHI', 'Predicted PHI', well_name='WellB#',top=3100, bot=3200)